# CARGA DE LIBRERIAS

In [1]:
import download_and_store


# DESCARGA DE LOS DATOS

In [2]:
download_and_store.initialize_database()
cats = download_and_store.download_data()
download_and_store.upload_to_db(cats)

Todo listo. Base de datos en: /Users/guille/Documents/LASALLE/MASTER/MD005/Entregas/XGBOOST/database/quickdraw.db
